In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, PolynomialFeatures
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s6e2/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/playground-series-s6e2/test.csv', index_col=0)
sample = pd.read_csv('/kaggle/input/playground-series-s6e2/sample_submission.csv')

target = 'Heart Disease'

# EDA

# Visuals

In [3]:
# fig = plt.figure(figsize=(10, 14))
# for index, col in enumerate(numeric_cols):
#     plt.subplot(6, 3, index+1)
#     sns.histplot(x=col, data=numeric_.dropna(), kde=True)
# fig.tight_layout(pad=1.0)

In [4]:
# fig = plt.figure(figsize=(10, 14))
# for index, col in enumerate(numeric_cols):
#     plt.subplot(6, 3, index+1)
#     sns.boxplot(y=col, data=numeric_.dropna())
# fig.tight_layout(pad=1.0)

In [5]:
# fig = plt.figure(figsize=(10, 14))
# for index, col in enumerate(cat_cols):
#     plt.subplot(7, 2, index+1)
#     sns.countplot(y=col, data=EDA_df.dropna())
# fig.tight_layout(pad=1.0)

In [6]:
# fig = plt.figure(figsize=(10, 14))
# for index, col in enumerate(EDA_df):
#     plt.subplot(7, 3, index+1)
#     sns.regplot(x=col, y='Heart Disease', data=EDA_df, ci=None, logistic=True)
# fig.tight_layout(pad=1.0)

## Bi-Variance

In [7]:
# from matplotlib import colormaps
# list(colormaps)

In [8]:
# # EDA_df['Heart Disease'] = EDA_df['Heart Disease'].map(MAP)
# corr = EDA_df.corr()
# sns.heatmap(corr, mask=corr<0.5, cmap='Blues')

## Data Cleaning

In [9]:
#TODO
# Maybe delete 'BP' AND 'Max HR'
# Get rid of outliers
# Normalize the data to fit the assumption of LogisticRegression

# Feature generation

In [10]:
# EDA_df['(Ves + Thall) / sqrt(MaxHR)'] = (EDA_df['Number of vessels fluro'] + EDA_df['Thallium']) / np.sqrt(EDA_df['Max HR'])

In [11]:
# EDA_df['Heart Disease'] = EDA_df['Heart Disease'].map(MAP)
# EDA_df.drop('(Ves + Thall)*2 / sqrt(MaxHR)', axis=1, inplace=True)

In [12]:
# # Too much data concentrated in 1 point
# dropping_columns = ['ST depression', 'FBS over 120', 'BP', 'Max HR', 'Thallium']
# EDA_df.drop(dropping_columns, axis=1, inplace=True)

In [13]:
# SUCCESS = 1 # 1 or 0 if EDA is okay, else raw data
# if SUCCESS or MODE == 'SUBMIT':
#     EDA_df['Heart Disease'] = EDA_df['Heart Disease'].map(MAP_alt)
#     train_df = EDA_df

# Doing data prep

In [14]:
EDA_df = train.copy()

cat_cols = ['Sex', 'Chest pain type', 'FBS over 120', 'EKG results', 'Exercise angina', 'Slope of ST', 'Number of vessels fluro', 'Thallium', target]

numeric_ = EDA_df.drop(cat_cols, axis=1)
cat_ = EDA_df[cat_cols]

In [15]:
# Outliears
def drop_outliears(df, col, inplace=False):
    if not inplace:
        df = df.copy()
        
    Q1 = np.percentile(df[col], 25)
    Q3 = np.percentile(df[col], 75)
    IQR = Q3 - Q1

    print("RANGE: [ " + str(Q1 - IQR * 1.5) + " ; " + str(Q3 + IQR * 1.5) +" ]")
    df = df[df[col] >= Q1 - IQR * 1.5]
    df = df[df[col] <= Q3 + IQR * 1.5]
    return df

# Drop outliears
for col in train.drop(cat_cols, axis=1):
    EDA_df = drop_outliears(EDA_df, col)

RANGE: [ 30.0 ; 78.0 ]
RANGE: [ 90.0 ; 170.0 ]
RANGE: [ 154.0 ; 338.0 ]
RANGE: [ 106.0 ; 202.0 ]
RANGE: [ -2.0999999999999996 ; 3.4999999999999996 ]


# Model

In [16]:
from xgboost import XGBClassifier
# model = LogisticRegression(C=0.1, max_iter=100000)
model = XGBClassifier(
    subsample=0.6,
    reg_lambda=10, 
    reg_alpha=0, 
    n_estimators=1000, 
    min_child_weight=3, 
    max_depth=2, 
    learning_rate=0.1, 
    gamma=0.5, 
    colsample_bytree=1.0
)

# Eval

In [17]:
# train.copy()
train_df = EDA_df
X, y = train_df.drop(target, axis=1), train_df[target]

# Target encoding
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(y)

In [18]:
# from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

# param_dist = {
#     "n_estimators": [200, 300, 500, 800, 1000],
#     "max_depth": [2, 3, 4, 5, 6],
#     "learning_rate": [0.01, 0.03, 0.05, 0.1],
#     "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
#     "colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],
#     "min_child_weight": [1, 3, 5, 7],
#     "gamma": [0, 0.5, 1, 2, 5],
#     "reg_alpha": [0, 0.5, 1, 5, 10],
#     "reg_lambda": [1, 5, 10, 20]
# }

# k=5
# kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# random_search = RandomizedSearchCV(
#     estimator=model,
#     param_distributions=param_dist,
#     n_iter=50,                  # number of parameter combinations to try
#     scoring="roc_auc",
#     cv=kf,
#     verbose=2,
#     random_state=42,
#     n_jobs=-1
# )

# random_search.fit(X, y)

In [19]:
# print("Best ROC-AUC:", random_search.best_score_)
# print("Best parameters:", random_search.best_params_)

In [20]:
# from sklearn.metrics import RocCurveDisplay
# from tqdm.auto import tqdm

# cv_scores = []

# for i, (train_index, test_index) in tqdm(enumerate(kf.split(X, y))):
#     print(f"Fold {i}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")

#     x_train, x_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]
    
#     # # Testing pol features
#     # pol = PolynomialFeatures(1)
#     # x_train = pol.fit_transform(x_train)
#     # x_test = pol.transform(x_test)
    
#     # # Scaling
#     # from sklearn.preprocessing import StandardScaler
    
#     # scaler = StandardScaler()
#     # x_train = scaler.fit_transform(x_train)
#     # x_test = scaler.transform(x_test)
    
#     model.fit(x_train, y_train)
#     y_pred = model.predict_proba(x_test)[:, 1]

#     cv_scores.append(roc_auc_score(y_test, y_pred))
#     print(f"  Model accuracy={roc_auc_score(y_test, y_pred)}")

# print(f"Average of CV ({k} folds)={sum(cv_scores)/k}")

In [21]:
# # Testing pol features
# pol = PolynomialFeatures(2)
# X = pol.fit_transform(X)
# # Test 
# test = pol.transform(test)

# # Scaling
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# # Test scaling
# test = scaler.transform(test)

model.fit(X, y)
y_pred = model.predict_proba(test)[:, 1]

# Submition

In [22]:
y_pred = model.predict(test)

In [23]:
sub_df = pd.DataFrame({'id': sample['id'], target: y_pred}, columns=['id', target])
sub_df.to_csv('submission.csv', index=False)